In [21]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
import numpy as np
import re

In [6]:
# Wczytanie modelu SpaCy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
nlp = spacy.load('en_core_web_md')

# Zadanie 1

Przeanalizować, czy pojawia się sarkazm i ironia w toksycznych komentarzach, szukając niezgodności między dosłownym znaczeniem a kontekstem emocjonalnym. Wykorzystać wektoryzację, aby analizować podobieństwo semantyczne komentarzy do przeciwstawnych emocji.

In [9]:
# Wczytanie danych
df = pd.read_csv('sample.csv')

In [22]:
# Tworzenie wektorów emocji
positive_emotion = nlp("joy, happiness, delight, love").vector
negative_emotion = nlp("anger, hatred, sadness, fear").vector

In [23]:
# Listy pozytywnych i negatywnych słów
positive_words = {"happy", "joy", "delight", "love", "exciting", "good"}
negative_words = {"angry", "hate", "bad", "sad", "awful", "terrible"}

In [26]:
def detect_sarcasm(text):
    # Wektoryzacja tekstu za pomocą modelu SpaCy
    doc = nlp(text)
    doc_vector = doc.vector

    # Ocena sentymentu całkowitego przy użyciu TextBlob
    blob = TextBlob(text)
    sentiment_polarity = blob.sentiment.polarity  # Polaryzacja sentymentu (-1 oznacza negatyw, +1 oznacza pozytyw)

    # Liczenie liczby pozytywnych i negatywnych słów w tekście
    pos_word_count = sum(1 for token in doc if token.lemma_ in positive_words)
    neg_word_count = sum(1 for token in doc if token.lemma_ in negative_words)

    # Obliczenie podobieństwa między wektorem komentarza a wektorami emocji
    pos_sim = cosine_similarity([doc_vector], [positive_emotion])[0][0]
    neg_sim = cosine_similarity([doc_vector], [negative_emotion])[0][0]

    # Wykrywanie potencjalnego sarkazmu na podstawie różnych wskaźników:
    # 1. Sprawdzamy, czy sentyment ogólny jest pozytywny, ale przeważa słownictwo negatywne (lub odwrotnie)
    if sentiment_polarity >= 0 and neg_word_count > pos_word_count:
        return 'Prawdopodobny sarkazm'  # Przypadek, gdy komentarz jest pozytywny, ale pełen negatywnych słów
    elif sentiment_polarity < 0 and pos_word_count > neg_word_count:
        return 'Prawdopodobny sarkazm'  # Przypadek, gdy komentarz jest negatywny, ale pełen pozytywnych słów

    # 2. Sprawdzamy, czy podobieństwo do emocji negatywnych jest większe niż do pozytywnych
    elif neg_sim > pos_sim:
        return 'Możliwy sarkazm lub ironia'  # Komentarz jest semantycznie bliższy emocjom negatywnym

    # 3. Jeśli nie wykryto żadnych sprzeczności ani podobieństwa do negatywnych emocji, przyjmujemy brak sarkazmu
    else:
        return 'Brak sarkazmu'

In [27]:
df['sarcasm_detection'] = df['comment_text'].apply(detect_sarcasm)
print(df[['comment_text', 'sarcasm_detection']].head())

                                        comment_text  \
0  What a breathe of fresh air to have someone wh...   
1  Your jewish friends were the ones who told you...   
2  Possible collusion by Trump and his affiliates...   
3  Exactly.  We need a % of GDP spending cap at t...   
4  By your own comment, even if some of them vote...   

            sarcasm_detection  
0               Brak sarkazmu  
1  Możliwy sarkazm lub ironia  
2  Możliwy sarkazm lub ironia  
3               Brak sarkazmu  
4  Możliwy sarkazm lub ironia  


# Zadanie 2

Dla książek Anna Karenina oraz Jane Eyre - Wyodrębnić opisy i dialogi wybranych bohaterów, np. Anny, Aleksieja, Jane i Edwarda. Obliczyć podobieństwa semantyczne między bohaterami i określić, jak różne są ich osobowości.

In [15]:
with open('anna_karenina.txt', 'r', encoding='utf-8') as f:
    anna_text = f.read()

with open('jane_eyre.txt', 'r', encoding='utf-8') as f:
    jane_text = f.read()

In [16]:
def extract_descriptions(text, character_name):
    sentences = [sent for sent in text.split('.') if character_name in sent]
    doc = nlp(' '.join(sentences))
    return doc.vector

In [17]:
anna_vector = extract_descriptions(anna_text, "Anna")
alexei_vector = extract_descriptions(anna_text, "Alexei")
jane_vector = extract_descriptions(jane_text, "Jane")
edward_vector = extract_descriptions(jane_text, "Edward")

In [18]:
anna_jane_sim = cosine_similarity([anna_vector], [jane_vector])[0][0]
alexei_edward_sim = cosine_similarity([alexei_vector], [edward_vector])[0][0]

In [19]:
print(f'Podobieństwo między Anną i Jane: {anna_jane_sim}')

Podobieństwo między Anną i Jane: 0.9419167637825012


In [20]:
print(f'Podobieństwo między Aleksiejem i Edwardem: {alexei_edward_sim}')

Podobieństwo między Aleksiejem i Edwardem: 0.7323974967002869
